In [ ]:
# -*- coding: utf-8 -*-
# جلسه ۸ - تمرین عملی: مقایسه مدل CNN با Dropout و بدون Dropout روی MNIST

# اگر لازم بود (معمولاً در کولب نصب هست):
# !pip install torch torchvision matplotlib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time

# -----------------------------
# 1) آماده‌سازی دستگاه (CPU/GPU)
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# -----------------------------
# 2) لود دیتاست MNIST
# -----------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # میانگین و انحراف معیار MNIST
])

train_dataset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# -----------------------------
# 3) تعریف مدل CNN با پارامتر Dropout
# -----------------------------
class SimpleCNN(nn.Module):
    def __init__(self, dropout_rate=0.0):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # 28x28 -> 28x28
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 28x28 -> 14x14

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 14x14 -> 14x14
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 14x14 -> 7x7
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),   # ⭐ این‌جا Dropout
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# -----------------------------
# 4) توابع آموزش و ارزیابی
# -----------------------------
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def train_model(dropout_rate, num_epochs=8, lr=1e-3):
    print(f"\n===== آموزش مدل با Dropout = {dropout_rate} =====")
    model = SimpleCNN(dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    history = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    start_time = time.time()
    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        test_loss, test_acc = evaluate(model, test_loader, criterion)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["test_loss"].append(test_loss)
        history["test_acc"].append(test_acc)

        print(f"Epoch {epoch:02d} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.3f} | "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.3f}")

    total_time = time.time() - start_time
    print(f"زمان کل آموزش: {total_time:.1f} ثانیه")
    return model, history

# -----------------------------
# 5) آموزش دو مدل: بدون Dropout و با Dropout
# -----------------------------
NUM_EPOCHS = 8

model_nodrop, hist_nodrop = train_model(dropout_rate=0.0, num_epochs=NUM_EPOCHS)
model_drop,  hist_drop  = train_model(dropout_rate=0.5, num_epochs=NUM_EPOCHS)

# -----------------------------
# 6) رسم نمودارهای مقایسه‌ای
# -----------------------------
epochs = np.arange(1, NUM_EPOCHS + 1)

plt.figure(figsize=(12,5))

# دقت
plt.subplot(1,2,1)
plt.plot(epochs, hist_nodrop["train_acc"], label="Train (No Dropout)")
plt.plot(epochs, hist_nodrop["test_acc"],  label="Test (No Dropout)")
plt.plot(epochs, hist_drop["train_acc"],   label="Train (Dropout 0.5)", linestyle="--")
plt.plot(epochs, hist_drop["test_acc"],    label="Test (Dropout 0.5)", linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("مقایسه دقت مدل با و بدون Dropout")
plt.legend()
plt.grid(True)

# لا‌س
plt.subplot(1,2,2)
plt.plot(epochs, hist_nodrop["train_loss"], label="Train (No Dropout)")
plt.plot(epochs, hist_nodrop["test_loss"],  label="Test (No Dropout)")
plt.plot(epochs, hist_drop["train_loss"],   label="Train (Dropout 0.5)", linestyle="--")
plt.plot(epochs, hist_drop["test_loss"],    label="Test (Dropout 0.5)", linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("مقایسه Loss مدل با و بدون Dropout")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# در انتها، آخرین دقت تست هر دو مدل را چاپ می‌کنیم
print("\nنتیجه نهایی:")
print(f"Test Accuracy (No Dropout):  {hist_nodrop['test_acc'][-1]:.3f}")
print(f"Test Accuracy (Dropout 0.5): {hist_drop['test_acc'][-1]:.3f}")


Device: cpu


100%|██████████| 9.91M/9.91M [00:00<00:00, 19.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 427kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.96MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.35MB/s]



===== آموزش مدل با Dropout = 0.0 =====
Epoch 01 | Train Loss: 0.1653, Train Acc: 0.951 | Test Loss: 0.0657, Test Acc: 0.978
